In [54]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import pandas as pd
from e2e_system.reproder import _ground_truth
import os


In [55]:
exp_main_dir = "data/paperswithcode/evaluated/hierarchical"
exp_name = "20230525-232040"
exp_dir = os.path.join(exp_main_dir, exp_name)


In [73]:
eval_direct_classification = pd.read_csv(
    os.path.join(exp_dir, 'eval_direct_classification.csv'))
eval_weighted_score = pd.read_csv(
    os.path.join(exp_dir, 'eval_weighted_score.csv'))
eval_direct_coeff = pd.read_csv(
    os.path.join(exp_dir, 'eval_direct_coeff.csv'))


In [74]:
eval_direct_classification.score = eval_direct_classification.score.apply(
    lambda x: (x*0.16) + 0.04)


In [68]:
eval_direct_coeff.corr(numeric_only=True).reprod_score.score


0.027826654043232323

In [77]:
eval_direct_classification

,url,stars,introduction,requirements,pretrained_model,evaluation,training,results,score,reprod_score
0,deepmind/lab,6082,True,True,False,True,True,False,0.68,0.36
1,zihangdai/xlnet,5114,True,False,True,True,True,True,0.84,0.20
2,tensorflow/tpu,3120,True,False,False,False,False,False,0.20,0.20
3,tensorflow/lingvo,1914,True,True,False,False,True,False,0.52,0.20
4,facebookresearch/XLM,1913,True,True,True,True,True,True,1.00,0.36
...,...,...,...,...,...,...,...,...,...,...
95,fkunstner/limitations-empirical-fisher,26,True,True,False,True,False,False,0.52,0.20
96,max-andr/provably-robust-boosting,26,True,True,False,True,True,False,0.68,0.20
97,PwnerHarry/Stronger_GCN,26,True,True,False,False,True,True,0.68,0.20
98,PKU-AI-Edge/FEN,23,True,False,False,False,False,False,0.20,0.20


In [76]:
eval_direct_classification.corr(numeric_only=True).reprod_score.score


0.02805766419108578

In [60]:
eval_weighted_score.corr(numeric_only=True).reprod_score.score


0.24337357414285368

In [61]:
accuracy = eval_direct_classification[eval_direct_classification.score ==
                                      eval_direct_classification.reprod_score].shape[0] / eval_direct_classification.shape[0]
print(
    f"Accuracy of direct classification: {round(accuracy, 3)}")


Accuracy of direct classification: 0.0


In [62]:
def analyze_reprod_score(df, type):
    df = df[["score", "reprod_score"]].describe()
    df["diff"] = df["reprod_score"] - df["score"]
    df = df.T.round(3)
    df.set_index([[type, type, type], [
        "true", "pred", "diff"]], inplace=True)
    return df


In [63]:
type_1_df = analyze_reprod_score(eval_direct_classification, "type 1")
type_2_df = analyze_reprod_score(eval_weighted_score, "type 2")
type_3_df = analyze_reprod_score(eval_direct_coeff, "type 3")


In [66]:
reprod_score_analysis = pd.concat(
    [type_1_df,type_2_df ,type_3_df])
reprod_score_analysis.drop(columns=["count"], inplace=True)


In [67]:
reprod_score_analysis.style.highlight_max(subset=pd.IndexSlice[reprod_score_analysis.index.isin([
                                          "diff"], level=1), :], color="green", axis=0).format("{:.3f}")
